In [ ]:
import pandas as pd
import re
import os
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
desired_columns = ['url', 'createdAt', 'id', 'viewCount','lang', 'author__createdAt', 'author__location', 'text']
cities = [
    'San Salvador', 'Soyapango', 'Santa Ana', 'San Miguel', 'Mejicanos', 'Santa Tecla',
    'Apopa', 'Delgado', 'Sonsonate', 'San Marcos', 'Usulután', 'Cojutepeque', 'Cuscatancingo',
    'San Vicente', 'Zacatecoluca', 'San Martín, El Salvador', 'Ilopango', 'Ahuachapán',
    'Antiguo Cuscatlán', 'Chalchuapa'
]

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'#\w+', '', text)  # Remove hashtags and the words following them
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    # text = re.sub(r'[\U00010000-\U0010FFFF]', '', text)  # Remove emojis
    return text

def check_location(location):
    if 'El Salvador' in location:
        return 'True'
    for city in cities:
        if city in location:
            return 'True'
    return 'False'

def process_file(input_file, output_dir):
    try:
        output_file = os.path.join(output_dir, os.path.basename(input_file).replace('replies', 'processed').replace('.xlsx', '.csv'))
        if os.path.exists(output_file):
            print(f"File {output_file} already processed. Skipping.")
            return

        comments_df = pd.read_excel(input_file, sheet_name='responses')
        desired_columns = ['url', 'createdAt', 'id', 'viewCount', 'lang', 'author__createdAt', 'author__location', 'text']
        comments_df = comments_df[[col for col in comments_df.columns if col in desired_columns]]

        date_format = '%a %b %d %H:%M:%S +0000 %Y'
        comments_df.loc[:, 'createdAt'] = pd.to_datetime(comments_df['createdAt'], format=date_format).astype(int) // 10**9

        comments_df = comments_df[comments_df['lang'] == 'es']
        comments_df.loc[:, 'author__location'] = comments_df['author__location'].fillna('')
        comments_df.loc[:, 'author__location'] = comments_df['author__location'].apply(check_location)
        comments_df.rename(columns={
            "author__location": "isComment_self__decalred__location__El_Salvador",
            'createdAt': 'comment_createdAt',
            'url': 'comment_url',
            'id': 'comment_id',
            'viewCount': 'comment_viewCount',
            'lang': 'comment_lang',
            'author__createdAt': 'comment_author__createdAt',
            'text': 'comment_text'
        }, inplace=True)

        comments_df = comments_df.dropna(subset=['comment_text'])
        comments_df = comments_df.assign(comment_processed_text=comments_df['comment_text'].apply(clean_text))

        post_df = pd.read_excel(input_file, sheet_name='initial_tweet')
        # print(post_df)
        post_df = post_df[[col for col in post_df.columns if col in desired_columns]]

        # convert createdAt to unix timestamp
        post_df.loc[:, 'createdAt'] = pd.to_datetime(post_df['createdAt'], format=date_format).astype(int) // 10**9

        post_df = post_df[post_df['lang'] == 'es']
        post_df.loc[:, 'author__location'] = post_df['author__location'].fillna('')
        post_df.loc[:, 'author__location'] = post_df['author__location'].apply(check_location)
        post_df.rename(columns={
            "author__location": "isPost_self__decalred__location__El_Salvador",
            'createdAt': 'post_createdAt',
            'url': 'post_url',
            'id': 'post_id',
            'viewCount': 'post_viewCount',
            'lang': 'post_lang',
            'author__createdAt': 'post_author__createdAt',
            'text': 'post_text'
        }, inplace=True)

        # print(post_df)
        post_df = post_df.dropna(subset=['post_text'])
        post_df = post_df.assign(post_processed_text=post_df['post_text'].apply(clean_text))

        if post_df.empty:
            # print(post_df)
            raise ValueError("Post DataFrame is empty after processing")

        merged_df = comments_df.assign(**{col: post_df.iloc[0][col] for col in post_df.columns})

        merged_df.to_csv(output_file, index=False)

    except Exception as e:
        print(f"Error processing file {input_file}: {e}")

def process_all_files(input_dir, output_dir):
    files = [f for f in os.listdir(input_dir) if f.endswith('.xlsx')]
    for file_name in tqdm(files, desc="Processing files"):
        input_file = os.path.join(input_dir, file_name)
        process_file(input_file, output_dir)

In [ ]:
input_directory = "/content/drive/MyDrive/Data_Irex/Data: RAW/Twitter/Comments"
output_directory = "/content/drive/MyDrive/Data_Irex/Data: Preprocessing/Twitter/Processed"
process_all_files(input_directory, output_directory)

Processing files:   1%|          | 1/148 [00:00<00:18,  7.78it/s]

Error processing file /content/drive/MyDrive/Data_Irex/Data: RAW/Twitter/Comments/nayibbukele_X_V3.2_replies_1776733270186545406.xlsx: Post DataFrame is empty after processing


Processing files:   2%|▏         | 3/148 [00:00<00:33,  4.27it/s]

Error processing file /content/drive/MyDrive/Data_Irex/Data: RAW/Twitter/Comments/nayibbukele_X_V3.2_replies_1783714043573702940.xlsx: Cannot save file into a non-existent directory: '/content/drive/MyDrive/Data_Irex/Data: Preprocessing/Twitter/Processed'
Error processing file /content/drive/MyDrive/Data_Irex/Data: RAW/Twitter/Comments/nayibbukele_X_V3.2_replies_1775699870231224819.xlsx: Cannot save file into a non-existent directory: '/content/drive/MyDrive/Data_Irex/Data: Preprocessing/Twitter/Processed'


Processing files:   3%|▎         | 5/148 [00:00<00:23,  6.03it/s]

...

Error processing file /content/drive/MyDrive/Data_Irex/Data: RAW/Twitter/Comments/ysuca91siete_X_V3.2_replies_1786046500583190702.xlsx: Cannot save file into a non-existent directory: '/content/drive/MyDrive/Data_Irex/Data: Preprocessing/Twitter/Processed'
Error processing file /content/drive/MyDrive/Data_Irex/Data: RAW/Twitter/Comments/andresguzm_X_V3.2_replies_1727067358349414796.xlsx: Cannot save file into a non-existent directory: '/content/drive/MyDrive/Data_Irex/Data: Preprocessing/Twitter/Processed'
Error processing file /content/drive/MyDrive/Data_Irex/Data: RAW/Twitter/Comments/andresguzm_X_V3.2_replies_1743277817154916369.xlsx: Cannot save file into a non-existent directory: '/content/drive/MyDrive/Data_Irex/Data: Preprocessing/Twitter/Processed'
Error processing file /content/drive/MyDrive/Data_Irex/Data: RAW/Twitter/Comments/andresguzm_X_V3.2_replies_1760607082376519973.xlsx: Cannot save file into a non-existent directory: '/content/drive/MyDrive/Data_Irex/Data: Preprocessi

Processing files:  99%|█████████▊| 146/148 [00:14<00:00, 16.94it/s]

Error processing file /content/drive/MyDrive/Data_Irex/Data: RAW/Twitter/Comments/andresguzm_X_V3.2_replies_1769337928327614558.xlsx: Cannot save file into a non-existent directory: '/content/drive/MyDrive/Data_Irex/Data: Preprocessing/Twitter/Processed'
Error processing file /content/drive/MyDrive/Data_Irex/Data: RAW/Twitter/Comments/andresguzm_X_V3.2_replies_1743328759866851688.xlsx: Cannot save file into a non-existent directory: '/content/drive/MyDrive/Data_Irex/Data: Preprocessing/Twitter/Processed'
Error processing file /content/drive/MyDrive/Data_Irex/Data: RAW/Twitter/Comments/andresguzm_X_V3.2_replies_1599405529486409730.xlsx: Cannot save file into a non-existent directory: '/content/drive/MyDrive/Data_Irex/Data: Preprocessing/Twitter/Processed'
Error processing file /content/drive/MyDrive/Data_Irex/Data: RAW/Twitter/Comments/PulsoCiudadanos_X_V3.2_replies_1760148904681640184.xlsx: Cannot save file into a non-existent directory: '/content/drive/MyDrive/Data_Irex/Data: Preproce

Processing files: 100%|██████████| 148/148 [00:15<00:00,  9.77it/s]

Error processing file /content/drive/MyDrive/Data_Irex/Data: RAW/Twitter/Comments/andresguzm_X_V3.2_replies_1682728543069896705.xlsx: Cannot save file into a non-existent directory: '/content/drive/MyDrive/Data_Irex/Data: Preprocessing/Twitter/Processed'
